In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("~/Desktop/mario_kart.csv")

track_order = [
    "Mario Circuit",
    "Moo Moo Meadows",
    "Wario Stadium",
    "Z Rainbow Road"
]

character_order = [
    "Mario",
    "Peach",
    "Wario",
    "DK", 
    "Koopa Troopa"
]

player_order = [
    "Jose", 
    "Zaibaa", 
    "Angela",
    "Cris"
]

# Convert Track to a categorical variable with the specified order
df['Track'] = pd.Categorical(df['Track'], categories=track_order, ordered=True)
df['Character'] = pd.Categorical(df['Character'], categories=character_order, ordered=True)
df['Player'] = pd.Categorical(df['Player'], categories=player_order, ordered=True)

# Now sort the DataFrame by track order if needed
df = df.sort_values('Track')

track_baseline = (
    df[df['Set'] == 'SetBase']
    .groupby('Track')['Seconds']
    .mean()
    .to_dict()
)

# Apply normalized time = race time / baseline time for that track
df['NormalizedSeconds'] = df.apply(
    lambda row: row['Seconds'] / track_baseline.get(row['Track'], None)
    if track_baseline.get(row['Track'], None) else None,
    axis=1
)

track_baseline2 = (
    df[df['Set'] == 'SetBase']
    .groupby('Track')['Fastest Lap']
    .mean()
    .to_dict()
)

df['NormalizedFastestLap'] = df.apply(
    lambda row: row['Fastest Lap'] / track_baseline2.get(row['Track'], None)
    if track_baseline2.get(row['Track'], None) else None,
    axis=1
)

df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Get fastest race per track
winners_per_track = df.loc[df.groupby('Track')['Seconds'].idxmin()].copy()

# Optional: rename columns for clarity
winners_per_track = winners_per_track[['Track', 'Player', 'Set', 'Character', 'Seconds', 'Fastest Lap']]
winners_per_track.rename(columns={"Seconds": "WinningTime"}, inplace=True)

# Get the fastest character per player per track
fastest_char_per_player_track = df.loc[df.groupby(['Player', 'Track'])['Seconds'].idxmin()].copy()

# Keep relevant columns
fastest_char_per_player_track = fastest_char_per_player_track[['Player', 'Track', 'Set', 'Character', 'Seconds', 'NormalizedSeconds', 'Fastest Lap']]


In [ ]:
sns.set_theme(style="whitegrid")
sns.set_palette("Set2")
df

In [ ]:
winners_per_track

In [ ]:
fastest_char_per_player_track

In [ ]:
# Player performance by Set
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x="Character", y="Seconds", hue="Player")
plt.title("Player Performance per Character (Lower Seconds = Better)")
plt.ylabel("Total Time (Seconds)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x="Character", y="NormalizedSeconds", hue="Player")
plt.axhline(1.0, color='gray', linestyle='--', label='SetBase Average')
plt.title("Normalised Race Time by Character and Player")
plt.ylabel("Normalised Time (Mario Mean = 1.0)")
plt.xticks(rotation=0)
plt.legend()
plt.tight_layout()
plt.show() 

In [ ]:
# ✅ Line plot to see performance shifts:
plt.figure(figsize=(10, 6))
# sns.pointplot(data=df, x="Character", y="NormalizedSeconds", hue="Player", estimator="mean", errorbar="sd", dodge=0.3)
sns.stripplot(data=df, x="Character", y="NormalizedSeconds", hue="Player")
plt.title("Average Normalized Time per Character by Player")
plt.axhline(1.0, color='gray', linestyle='--')
plt.ylabel("Normalized Time (SetBase = 1.0)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.stripplot(data=df, x="Track", y="NormalizedSeconds", hue="Player")
plt.axhline(1.0, color='gray', linestyle='--', label='SetBase Average')
plt.title("Normalized Race Time by Track and Player")
plt.ylabel("Normalized Time (Mario Mean = 1.0)")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show() 

In [ ]:
plt.figure(figsize=(10, 6))
sns.pointplot(data=df, x="Track", y="NormalizedSeconds", hue="Player", estimator="median", dodge=0.3)
plt.title("Median Normalised Time per Character by Track")
plt.axhline(1.0, color='gray', linestyle='--')
plt.ylabel("Normalised Time (SetBase = 1.0)")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
#✅ 4. FacetGrid: One plot per Player showing Set performance
g = sns.FacetGrid(df, col="Track", col_wrap=4, height=4, sharey=True)
g.map(sns.boxplot, "Character", "NormalizedSeconds", order=sorted(df["Character"].unique()))
g.set_titles("{col_name}'s Performance")
g.set_axis_labels("Character", "NormalizedSeconds")
plt.tight_layout()
plt.show()

In [ ]:
#✅ 4. FacetGrid: One plot per Player showing Set performance
g = sns.FacetGrid(df, col="Track", col_wrap=4, height=4, sharey=True)
g.map(sns.boxplot, "Player", "Seconds", order=sorted(df["Player"].unique()))
g.set_titles("{col_name}'s Performance")
g.set_axis_labels("Player", "Seconds")
plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(df, hue="Player", vars=["NormalizedSeconds", "NormalizedFastestLap"], kind="scatter")

In [ ]:
pivot = fastest_char_per_player_track.pivot(index='Track', columns='Player', values='Character')
print(pivot)

In [ ]:
plt.figure(figsize=(10, 6))
sns.stripplot(
    data=fastest_char_per_player_track,
    x='Track',
    y='NormalizedSeconds',
    hue='Character',
    dodge=True,
    jitter=True
)
plt.title("Best Character per Player per Track")
plt.xticks(rotation=0, ha='center')
plt.ylabel("Fastest Time (Seconds)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

print(pivot)

In [ ]:
player = 'Angela'
measurement = 'Seconds'
df_player = df[df['Player'] == player].copy()

# Create a subplot (1 row, 4 columns)
fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=True)

# Plot one subplot per track
for i, track in enumerate(track_order):
    ax = axes[i]
    df_track = df_player[df_player['Track'] == track]

    # Skip if no data for that track
    if df_track.empty:
        ax.set_title(track + "\n(no data)")
        ax.axis('off')
        continue

    sns.barplot(
        data=df_track,
        x='Character',
        y=measurement,
        ax=ax,
        palette='muted'
    )

    ax.set_title(track)
    ax.set_xlabel('')
    ax.set_ylabel(measurement if i == 0 else '')  # Only leftmost plot gets y-axis label
    # ax.set_ylim(100, df_player[measurement].max() + 5)
    ax.tick_params(axis='x', rotation=45)

fig.suptitle(f"Performance per Character for {player}", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# Create horizontal line plots

# Example ranges for left and right ventricle (replace with your actual values)
lv_ranges = {'abnormal low': (0, 93), 'normal': (109, 218), 'abnormal high': (232, 400)}
rv_ranges = {'abnormal low': (0, 99), 'normal': (124, 248), 'abnormal high': (260, 400)}

# C_M	Vol_LV	179.6324615	36.31353599
# C_M	Vol_RV	188.6540769	25.74344301
# C_F	Vol_LV	139.7238923	32.19235611
# C_F	Vol_RV	139.6913923	34.84204361
# HF_M	Vol_LV	289.08675	88.08042718
# HF_M	Vol_RV	227.6365833	60.16697976
# HF_F	Vol_LV	183.0358	60.04327315
# HF_F	Vol_RV	136.7402333	30.79667327

lv_means = [179.6324615, 289.08675]
lv_means_females = [139.7238923, 183.0358]

lv_stds = [36.31353599, 88.08042718]
lv_stds_females = [32.19235611, 60.04327315]

rv_means = [188.6540769, 227.6365833]
rv_means_females =[139.6913923, 136.7402333]

rv_std = [25.74344301, 60.16697976]
rv_std_females = [34.84204361, 30.79667327 ]

fig, axes = plt.subplots(1, 2, figsize=(14, 3), sharey=True)

for ax, (title, ranges) in zip(axes, [('Left Ventricle', lv_ranges), ('Right Ventricle', rv_ranges)]):
    y = 0  # All bars on the same line
    for label, (low, high) in ranges.items():
        ax.barh(y, high - low, left=low, color={
            'abnormal low': 'lightblue',
            'normal': 'lightgreen',
            'abnormal high': 'salmon'
        }[label], edgecolor='black', height=0.6, label=label)
    # Example: Calculate mean and std from your data (replace with your ac tual columns)
    if title == 'Left Ventricle':
        ax.errorbar(lv_means, [-0.1, -0.05], xerr=lv_stds, fmt='o', color='black', label='Mean ± SD')
        # ax.errorbar(lv_means_females, [0.05, 0.1], xerr=lv_stds_females, fmt='o', color='black', label='Mean ± SD')
    else:
        ax.errorbar(rv_means, [-0.1, -0.05], xerr=rv_std, fmt='o', color='black', label='Mean ± SD')
        # ax.errorbar(rv_means_females, [0.05, 0.1], xerr=rv_std_females, fmt='o', color='black', label='Mean ± SD')

    ax.set_title(title)
    ax.set_xlabel('Measurement Value')
    ax.set_yticks([y])
    ax.set_yticklabels(['Ranges'])
    ax.legend(loc='upper right')

plt.tight_layout()
plt.show()
